In [ ]:
# 使用Prophet预测manning未来365天的页面流量
# 从2007年12月10日开始
import pandas as pd
from fbprophet import Prophet
import matplotlib.pyplot as plt
%matplotlib inline

# 读入数据集
df = pd.read_csv('./manning.csv')
#print(df.head())
#print(df.tail())

# 对节假日建模
# 将节日看成是一个正态分布，把活动期间当做波峰，lower_window 以及upper_window 的窗口作为扩散
playoffs = pd.DataFrame({
  'holiday': 'playoff',
  'ds': pd.to_datetime(['2008-01-13', '2009-01-03', '2010-01-16',
                        '2010-01-24', '2010-02-07', '2011-01-08',
                        '2013-01-12', '2014-01-12', '2014-01-19',
                        '2014-02-02', '2015-01-11', '2016-01-17',
                        '2016-01-24', '2016-02-07']),
  'lower_window': 0,
  'upper_window': 1,
})
superbowls = pd.DataFrame({
  'holiday': 'superbowl',
  'ds': pd.to_datetime(['2010-02-07', '2014-02-02', '2016-02-07']),
  'lower_window': 0,
  'upper_window': 1,
})
holidays = pd.concat((playoffs, superbowls))

# 拟合模型
#df['cap'] = 8.5 #给DataFrame加一列cap则启用饱和上限
#df['floor'] = 1.5 #设定下限
model = Prophet() #加上参数growth='logistic'则使用logistic模型(注意logistic模型必须设置cap)，参数changepoints=['2014-01-01']指定要拟合的突变点，参数holidays=holidays将节假日考虑进模型
model.fit(df)

# 提取时间列并延长预测时间，periods = 365 代表除历史数据的日期外再往后推 365 天
future = model.make_future_dataframe(periods=365)
#print(future.tail())
#future['cap'] = 8.5 # 设定未来的饱和上限
#future['floor'] = 1.5 #设定未来的下限

# 预测数据集
forecast = model.predict(future) #将延长后的时间列输入模型进行预测，会返回很多字段
#print(forecast.columns)
#print(forecast)
#forecast.tail()
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())

# 展示预测结果
model.plot(forecast)
plt.show()

# 预测的成分分析绘图，展示预测中的趋势、周效应和年度效应
model.plot_components(forecast)
print(forecast.columns)

# 通过 forecast 数据框，展示节假日效应
forecast[(forecast['playoff'] + forecast['superbowl']).abs() > 0][['ds', 'playoff', 'superbowl']][-10:]

# 可以使用 plot_forecast_component(从fbprophet.plot导入)来画出独立的节假日的成分
from fbprophet.plot import plot_forecast_component
model.plot_forecast_component(forecast, 'superbowl')

#标记显著的突变点
from fbprophet.plot import add_changepoints_to_plot
fig = model.plot(forecast)
# plt.gca()获得当前的Axes对象ax
# 获取显著的突变点的位置
a = add_changepoints_to_plot(fig.gca(), model, forecast)
print(model)